In [33]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
import itertools
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pickle
import random
import os
import h5py 
from os import walk
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import joblib
tf.random.set_seed(42)

In [34]:
#Store the cancer dataset in a pandas dataframe.
df_cancer = pd.read_csv(r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\tc_labels.csv',header=None)
#df.drop(2, inplace=True, axis=1)
df_cancer.rename(columns = {0:'Folder name', 1:'label'}, inplace = True)

In [35]:
#Get the classes of our folders
#Build the dataset
df_til = pd.read_csv(r'C:\Users\fabga\OneDrive\Desktop\actual data\til_labels_binary.csv',header=None)
#df_til.drop(2, inplace=True, axis=1)
df_til.rename(columns = {0:'Folder name', 1:'label'}, inplace = True)

In [46]:
#Extract several quantites from the Cancer dataset(e.g, number of WSIs, number of tiles, min,max,avg,std of the number of tiles)
number_of_patches = 0
number_of_0 = 0
number_of_1 = 0
number_of_cancer_patches = 0
number_of_healty_patches = 0
patches_list = []
negative_patches_list = []
positive_patches_list = []

j=0
for folder in os.listdir(r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\thesis_data_jpg' ):
    j+=1
    name = r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\thesis_data_jpg' + '/' + folder
    number_of_files = len([name for value in os.listdir(name) if os.path.isfile(name + '/' + value)])
    label = df_cancer.loc[df_cancer['Folder name']==os.path.basename(folder)]['label'].item()
    if label == '1':
        number_of_1 += 1
        number_of_cancer_patches += number_of_files
        positive_patches_list.append(number_of_files)
        label = 'TUMOR'
    elif label== '0':
        number_of_0 += 1
        number_of_healty_patches += number_of_files
        negative_patches_list.append(number_of_files)
        label = 'NORMAL'
    patches_list.append(number_of_files)
    print("In the folder {} we have {} images.Label is: {}".format(folder,number_of_files,label))
    number_of_patches += number_of_files
print("")    
print("")    

print("The total number of images is: {}".format(number_of_patches))
print(" {} folders have label = TUMOR".format(number_of_1))
print(" {} folders have label = NORMAL".format(number_of_0))
print("The total number of TUMOR patches is {}".format(number_of_cancer_patches))
print("The total number of NORMAL patches is {}".format(number_of_healty_patches))

print("The average number of patches for all folders is: {}".format(np.mean(np.array(patches_list))))
print("The average number of patches for TUMOR folders is: {}".format(np.mean(np.array(positive_patches_list))))
print("The average number of patches for NORMAL folders is: {}".format(np.mean(np.array(negative_patches_list))))

print("The patches std for all folders is: {}".format(np.std(np.array(patches_list))))
print("The patches std for TUMOR folders is: {}".format(np.std(np.array(positive_patches_list))))
print("The patches std for NORMAL folders is: {}".format(np.std(np.array(negative_patches_list))))

print("The max number of patches for all folders is: {}".format(max(patches_list)))
print("The max number of patches for TUMOR folders is: {}".format(max(positive_patches_list)))
print("The max number of patches for NORMAL folders is: {}".format(max(negative_patches_list)))

print("The min number of patches for all folders is: {}".format(min(patches_list)))
print("The min number of patches for TUMOR folders is: {}".format(min(positive_patches_list)))
print("The min number of patches for NORMAL folders is: {}".format(min(negative_patches_list)))


In the folder TCGA-2Z-A9JL-01A-01-TS1 we have 1904 images.Label is: TUMOR
In the folder TCGA-A3-3362-01A-01-BS1 we have 153 images.Label is: TUMOR
In the folder TCGA-A3-3362-01A-02-BS2 we have 236 images.Label is: TUMOR
In the folder TCGA-A3-A6NL-11A-01-TS1 we have 552 images.Label is: NORMAL
In the folder TCGA-A3-A8CQ-01A-01-TS1 we have 944 images.Label is: TUMOR
In the folder TCGA-A4-A4ZT-11A-01-TS1 we have 150 images.Label is: NORMAL
In the folder TCGA-A4-A57E-11A-01-TS1 we have 373 images.Label is: NORMAL
In the folder TCGA-B0-5097-01A-01-BS1 we have 221 images.Label is: TUMOR
In the folder TCGA-B0-5097-01A-01-TS1 we have 79 images.Label is: TUMOR
In the folder TCGA-B0-5113-01A-01-BS1 we have 19 images.Label is: TUMOR
In the folder TCGA-B0-5113-01A-01-TS1 we have 23 images.Label is: TUMOR
In the folder TCGA-B0-5712-01A-01-BS1 we have 265 images.Label is: TUMOR
In the folder TCGA-B8-A54E-01A-01-TS1 we have 846 images.Label is: TUMOR
In the folder TCGA-B8-A54G-01A-01-TS1 we have 481 

In [44]:
#Count tiles and labels.
#Count the number of patches in each folder
number_of_patches = 0
number_of_0 = 0
number_of_1 = 0
number_of_til_patches = 0
number_of_notil_patches = 0
patches_list = []
negative_patches_list = []
positive_patches_list = []

j=0 
for folder in os.listdir(r'C:\Users\fabga\OneDrive\Desktop\actual data\thesis_data_jpg'):
    j+=1
    name = r'C:\Users\fabga\OneDrive\Desktop\actual data\thesis_data_jpg' + '/' + folder
    number_of_files = len([name for value in os.listdir(name) if os.path.isfile(name + '/' + value)])
    label = df_til.loc[df_til['Folder name']==os.path.basename(folder)]['label'].item()
    if label == 1:
        number_of_1 += 1
        number_of_til_patches += number_of_files
        positive_patches_list.append(number_of_files)
        label = 'TIL'
    elif label== 0:
        number_of_0 += 1
        number_of_notil_patches += number_of_files
        negative_patches_list.append(number_of_files)
        label = 'NON-TIL'
    patches_list.append(number_of_files)
    print("In the folder {} we have {} images.Label is: {}.".format(folder,number_of_files,label))
    number_of_patches += number_of_files
print("")
print("")

print("The total number of images is: {}".format(number_of_patches))
print(" {} folders have label = TIL".format(number_of_1))
print(" {} folders have label = NON-TIL".format(number_of_0))
print("The total number of TIL patches is {}".format(number_of_til_patches))
print("The total number of NON-TIL patches is {}".format(number_of_notil_patches))

print("The average number of patches for all folders is: {}".format(np.mean(np.array(patches_list))))
print("The average number of patches for TIL folders is: {}".format(np.mean(np.array(positive_patches_list))))
print("The average number of patches for NON-TIL folders is: {}".format(np.mean(np.array(negative_patches_list))))

print("The patches std for all folders is: {}".format(np.std(np.array(patches_list))))
print("The patches std for TIL folders is: {}".format(np.std(np.array(positive_patches_list))))
print("The patches std for NON-TIL folders is: {}".format(np.std(np.array(negative_patches_list))))

print("The max number of patches for all folders is: {}".format(max(patches_list)))
print("The max number of patches for TIL folders is: {}".format(max(positive_patches_list)))
print("The max number of patches for NON-TIL folders is: {}".format(max(negative_patches_list)))

print("The min number of patches for all folders is: {}".format(min(patches_list)))
print("The min number of patches for TIL folders is: {}".format(min(positive_patches_list)))
print("The min number of patches for NON-TIL folders is: {}".format(min(negative_patches_list)))


In the folder TCGA-A3-3343-01A-01-BS1 we have 19 images.Label is: TIL.
In the folder TCGA-A3-3343-01A-01-TS1 we have 29 images.Label is: TIL.
In the folder TCGA-A3-3346-01A-01-BS1 we have 511 images.Label is: TIL.
In the folder TCGA-A3-3346-01A-01-TS1 we have 639 images.Label is: TIL.
In the folder TCGA-A3-3347-01A-01-TS1 we have 476 images.Label is: TIL.
In the folder TCGA-A3-A6NL-11A-01-TS1 we have 552 images.Label is: NON-TIL.
In the folder TCGA-A3-A8CQ-01A-01-TS1 we have 944 images.Label is: NON-TIL.
In the folder TCGA-A4-A4ZT-01A-01-TS1 we have 1452 images.Label is: TIL.
In the folder TCGA-A4-A4ZT-11A-01-TS1 we have 150 images.Label is: NON-TIL.
In the folder TCGA-A4-A57E-11A-01-TS1 we have 373 images.Label is: NON-TIL.
In the folder TCGA-A4-A7UZ-01A-01-TS1 we have 442 images.Label is: TIL.
In the folder TCGA-AK-3425-01A-01-BS1 we have 320 images.Label is: TIL.
In the folder TCGA-AK-3425-01A-02-BS2 we have 96 images.Label is: TIL.
In the folder TCGA-B0-5706-01A-01-BS1 we have 293 

In [ ]:
####################################################################
####################################################################

In [ ]:
#Data augmentation layers:
#Rotation//flip

augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.3,seed=42),
    layers.RandomZoom(height_factor=(-0.3, +0.3), fill_mode = 'reflect',seed=42 ),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=42)]
)

#Resnet pretrained model:
model = ResNet50(weights="imagenet",input_shape=[224,224,3], include_top=False)

In [ ]:
#Extracts images from the folders (WSIs).
#Augments the images with the augmentation layer.
#Extract features from images through Resnet-50.
#Aggregates features through Max and Mean aggregation strategies.
#Returns two pandas dataframes with the aggregated featrues.

def augmented_batch_extraction(batch_size,image_path,folders_list):
    df_extraction_mean = pd.DataFrame([],columns=np.arange(100352))
    df_extraction_max = pd.DataFrame([],columns=np.arange(100352))
    
    j=0

    #for folder in os.listdir(image_path_training):
    for folder in folders_list:
        print("We are entering folder: {}".format(folder))
        folder_path = image_path + '\\' + folder 
        
        image_folder_list = []
        feature_list_batch = []
        for (dirpath, dirnames, filenames) in walk(folder_path):
            last_value = 0       
            if batch_size <= len(filenames):
                for _ in range(len(filenames)):
                    imagePath = folder_path + "\\" + filenames[_]

                    #Load images (and their augmentations) as arrays
                    image = img_to_array(load_img(imagePath, target_size=(224, 224)))
                    augmented_image = img_to_array(augmentation(image))
                    augmented_image_copy = augmented_image.copy()
            
                    #preprocess images
                    image = preprocess_input(image)
                    augmented_image_copy = preprocess_input(augmented_image_copy)
            
                    #add images to folder list
                    image_folder_list.append(image)
                    image_folder_list.append(augmented_image_copy)
            
                    if (_+1) % batch_size == 0:
                        image_array = np.array(image_folder_list[last_value:2*_+2])
                        image_features = model.predict(image_array, batch_size=batch_size)
                        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
                        feature_list_batch.append(image_features)
                        last_value = 2*_+2
                    elif _ == len(filenames)-1:
                        image_array = np.array(image_folder_list[last_value:2*len(filenames)])
                        image_features = model.predict(image_array, batch_size=batch_size)
                        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
                        feature_list_batch.append(image_features)
            
            else:
                for _ in range(len(filenames)):
                    #Load images (and their augmentations) as arrays
                    imagePath = folder_path + "\\" + filenames[_]
                    image = img_to_array(load_img(imagePath, target_size=(224, 224)))
                    augmented_image = img_to_array(augmentation(image))
                    augmented_image_copy = augmented_image.copy()
            
                    #preprocess images
                    image = preprocess_input(image)
                    augmented_image_copy = preprocess_input(augmented_image_copy)
            
                    #add images to folder list
                    image_folder_list.append(image)
                    image_folder_list.append(augmented_image_copy)
                
                image_array = np.array(image_folder_list)
                image_features = model.predict(image_array, batch_size=len(filenames))
                image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))#.tolist()
                feature_list_batch.append(image_features)
            feature_array = np.concatenate(feature_list_batch, axis=0)
            mean_list = np.mean(feature_array,axis=0)
            max_list = np.max(feature_array,axis=0)
            df_extraction_mean.loc[j] = mean_list
            df_extraction_max.loc[j] = max_list

            #data_list.append(feature_array)
            del mean_list
            del max_list
            del feature_array
        j+=1
    return(df_extraction_mean,df_extraction_max)

In [ ]:
#Extracts images from the folders (WSIs).
#Extract features from images through Resnet-50.
#Aggregates features through Max and Mean aggregation strategies.
#Returns two pandas dataframes with the aggregated featrues.

def batch_extraction(batch_size,image_path,folders_list):
    df_extraction_mean = pd.DataFrame([],columns=np.arange(100352))
    df_extraction_max = pd.DataFrame([],columns=np.arange(100352))

    
    j=0

    #for folder in os.listdir(image_path_training):
    for folder in folders_list:
        print("We are entering folder: {}".format(folder))
        folder_path = image_path + '\\' + folder 
        
        image_folder_list = []
        feature_list_batch = []
        for (dirpath, dirnames, filenames) in walk(folder_path):
            #print(filenames)
            last_value = 0       
            if batch_size <= len(filenames):
                #print("I am in the first condition!")
                for _ in range(len(filenames)):
                    #Load imagesas arrays
                    imagePath = folder_path + "\\" + filenames[_]
                    image = img_to_array(load_img(imagePath, target_size=(224, 224)))
            
                    #preprocess images
                    image = preprocess_input(image)
            
                    #add images to folder list
                    image_folder_list.append(image)
            
                    if (_+1) % batch_size == 0:
                        #print(_+1)
                        image_array = np.array(image_folder_list[last_value:_+1])
                        image_features = model.predict(image_array, batch_size=batch_size)
                        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
                        feature_list_batch.append(image_features)
                        last_value = _+1
                    elif _ == len(filenames)-1:
                        image_array = np.array(image_folder_list[last_value:2*len(filenames)])
                        image_features = model.predict(image_array, batch_size=batch_size)
                        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
                        feature_list_batch.append(image_features)          
            else:
                for _ in range(len(filenames)):
                    imagePath = folder_path + "\\" + filenames[_]
                    image = img_to_array(load_img(imagePath, target_size=(224, 224)))

                    #preprocess images
                    image = preprocess_input(image)
            
                    #add images to folder list
                    image_folder_list.append(image)   
                
                image_array = np.array(image_folder_list)
                image_features = model.predict(image_array, batch_size=len(filenames))
                image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
                feature_list_batch.append(image_features)

            feature_array = np.concatenate(feature_list_batch, axis=0)
            mean_list = np.mean(feature_array,axis=0)
            max_list = np.max(feature_array,axis=0)
            df_extraction_mean.loc[j] = mean_list
            df_extraction_max.loc[j] = max_list
            del mean_list
            del max_list
            del feature_array
        j+=1
    return(df_extraction_mean,df_extraction_max)

In [ ]:
#Logistic regression 10-fold crossvalidation
C_list = [0.0001,0.001,0.01,0.1,1,10,100,1000]

for c in C_list:
        print("c value is: {}".format(c))
        fig, [ax_max, ax_mean] = plt.subplots(1, 2, figsize=(18, 11))

        #Initialize lists to manage metrics.
        avg_precision_train_mean = []
        avg_precision_test_mean = []

        avg_recall_train_mean = []
        avg_recall_test_mean = []

        avg_f1_train_mean = []
        avg_f1_test_mean = []

        avg_score_train_mean = []
        avg_score_test_mean = []

        avg_precision_train_max = []
        avg_precision_test_max = []

        avg_recall_train_max = []
        avg_recall_test_max = []

        avg_f1_train_max = []
        avg_f1_test_max = []
        
        avg_score_train_max = []
        avg_score_test_max = []

        #Lists for ROC curves. 
        tprs_mean = [] 
        aucs_mean = []

        tprs_max = []
        aucs_max = []
        
        mean_fpr_max = np.linspace(0, 1, 100)
        mean_fpr_mean = np.linspace(0, 1, 100)

        logisticRegr_mean = LogisticRegression(C = c,max_iter = 500)
        logisticRegr_max = LogisticRegression(C = c,max_iter = 500)


        for i in range(10):
            _seed = i

    
            df_mean['labels'] = label_list
            df_max['labels'] = label_list


            y_mean = df_mean.pop('labels')
            X_mean = df_mean

            y_max = df_max.pop('labels')
            X_max = df_max
    
    
            #Create training and test sets for each fold.
            X_train_mean_index,X_test_mean_index,y_train_mean,y_test_mean = train_test_split(X_mean.index,y_mean,random_state =_seed, test_size=0.4)
            X_train_max_index,X_test_max_index,y_train_max,y_test_max = train_test_split(X_max.index,y_max,random_state =_seed,test_size=0.4)

    
            X_train_mean = X_mean.iloc[X_train_mean_index].to_numpy()
            X_test_mean = X_mean.iloc[X_test_mean_index].to_numpy()
            y_train_mean = y_train_mean.to_numpy()
            y_test_mean = y_test_mean.to_numpy()
    

            X_train_max = X_max.iloc[X_train_max_index].to_numpy()
            X_test_max = X_max.iloc[X_test_max_index].to_numpy()
            y_train_max = y_train_max.to_numpy()
            y_test_max = y_test_max.to_numpy()
        
    
            #MEAN AGGREGATION MODEL: perform logistic regression on mean-aggregated data.
            logisticRegr_mean.fit(X_train_mean, y_train_mean)
            y_score = logisticRegr_mean.fit(X_train_mean, y_train_mean).decision_function(X_test_mean)
            predictions_train_mean = logisticRegr_mean.predict(X_train_mean)
            predictions_test_mean = logisticRegr_mean.predict(X_test_mean)
    
            score_training_mean = logisticRegr_mean.score(X_train_mean, y_train_mean)
            f1_training_mean = f1_score(y_train_mean, predictions_train_mean, average='binary')
            precision_training_mean = precision_score(y_train_mean, predictions_train_mean, average='binary')
            recall_training_mean = recall_score(y_train_mean, predictions_train_mean, average='binary')
    
            score_test_mean = logisticRegr_mean.score(X_test_mean, y_test_mean)
            f1_test_mean = f1_score(y_test_mean, predictions_test_mean, average='binary')
            precision_test_mean =precision_score(y_test_mean, predictions_test_mean, average='binary')
            recall_test_mean = recall_score(y_test_mean, predictions_test_mean, average='binary')

            avg_precision_train_mean.append(precision_training_mean)
            avg_precision_test_mean.append(precision_test_mean)

            avg_recall_train_mean.append(recall_training_mean)
            avg_recall_test_mean.append(recall_test_mean)

            avg_f1_train_mean.append(f1_training_mean)
            avg_f1_test_mean.append(f1_test_mean)

            avg_score_train_mean.append(score_training_mean)
            avg_score_test_mean.append(score_test_mean)
    
            #collect data for mean aggregation ROC curve
            viz_mean = RocCurveDisplay.from_estimator(
                logisticRegr_mean,
                X_test_mean,
                y_test_mean,
                ax=ax_mean,
                name="ROC fold {}".format(i+1),
                alpha=0.3,
            )
    
            interp_tpr_avg = np.interp(mean_fpr_mean, viz_mean.fpr, viz_mean.tpr)
            interp_tpr_avg[0] = 0.0
            tprs_mean.append(interp_tpr_avg)
            aucs_mean.append(viz_mean.roc_auc)
    
    
            #MAX AGGREGATION MODEL: perform logistic regression on max-aggregated data.
            logisticRegr_max.fit(X_train_max, y_train_max)
            y_score = logisticRegr_max.fit(X_train_max, y_train_max).decision_function(X_test_max)
            predictions_train_max = logisticRegr_max.predict(X_train_max)
            predictions_test_max = logisticRegr_max.predict(X_test_max)

            score_training_max = logisticRegr_max.score(X_train_max, y_train_max)
            f1_training_max = f1_score(y_train_max, predictions_train_max, average='binary')
            precision_training_max = precision_score(y_train_max, predictions_train_max, average='binary')
            recall_training_max = recall_score(y_train_max, predictions_train_max, average='binary')

            score_test_max = logisticRegr_max.score(X_test_max, y_test_max)
            f1_test_max = f1_score(y_test_max, predictions_test_max, average='binary')
            precision_test_max =precision_score(y_test_max, predictions_test_max, average='binary')
            recall_test_max = recall_score(y_test_max, predictions_test_max, average='binary')

            avg_precision_train_max.append(precision_training_max)
            avg_precision_test_max.append(precision_test_max)

            avg_recall_train_max.append(recall_training_max)
            avg_recall_test_max.append(recall_test_max)

            avg_f1_train_max.append(f1_training_max)
            avg_f1_test_max.append(f1_test_max)

            avg_score_train_max.append(score_training_max)
            avg_score_test_max.append(score_test_max)


            #collect data for max aggregation ROC curve
            viz_max = RocCurveDisplay.from_estimator(
                logisticRegr_max,
                X_test_max,
                y_test_max,
                ax=ax_max,
                name="ROC fold {}".format(i+1),
                alpha=0.3,
            )
    

            interp_tpr_max = np.interp(mean_fpr_max, viz_max.fpr, viz_max.tpr)
            interp_tpr_max[0] = 0.0
            tprs_max.append(interp_tpr_max)
            aucs_max.append(viz_max.roc_auc)
    
    

        #Plot avg ROC curves (max aggregation)
        mean_tpr_max = np.mean(tprs_max, axis=0)
        mean_tpr_max[-1] = 1.0
        mean_auc_max = auc(mean_fpr_max, mean_tpr_max)
        std_auc_max = np.std(aucs_max)

        ax_max.plot(
            mean_fpr_max,
            mean_tpr_max,
            color="b",
            label=r"Max ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_max, std_auc_max),
            lw=2,
            alpha=0.8,
        )    

        ax_max.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_max.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
        )
        ax_max.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)

        #Plot avg ROC curves (mean aggregation)
        mean_tpr_mean = np.mean(tprs_mean, axis=0)
        mean_tpr_mean[-1] = 1.0
        mean_auc_mean = auc(mean_fpr_mean, mean_tpr_mean)
        std_auc_mean = np.std(aucs_mean)

        ax_mean.plot(
            mean_fpr_mean,
            mean_tpr_mean,
            color="b",
            label=r"Max ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_mean, std_auc_mean),
            lw=2,
            alpha=0.8,
        )    

        ax_mean.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_mean.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
        )
        ax_mean.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)


        ax_max.set_title("Receiver Operating Characteristic Curves, Max aggregation, Log. Regression.")
        ax_mean.set_title("Receiver Operating Characteristic Curves, Mean aggregation, Log. Regression.")

        ax_max.grid(linestyle="--")
        ax_mean.grid(linestyle="--")

        ########################################
        ########################################
        print("Avg score training sets mean: {}".format(np.mean(np.array(avg_score_train_mean))))
        print("Avg score test sets mean: {}".format(np.mean(np.array(avg_score_test_mean))))

        print("Avg precision training sets mean: {}".format(np.mean(np.array(avg_precision_train_mean))))
        print("Avg precision test sets mean: {}".format(np.mean(np.array(avg_precision_test_mean))))

        print("Avg recall training sets mean: {}".format(np.mean(np.array(avg_recall_train_mean))))
        print("Avg recall test sets mean: {}".format(np.mean(np.array(avg_recall_test_mean))))

        print("Avg F1 training sets mean: {}".format(np.mean(np.array(avg_f1_train_mean))))
        print("Avg F1 test sets mean: {}".format(np.mean(np.array(avg_f1_test_mean))))
        print("")
        print("")
        print("Std score training sets mean: {}".format(np.std(np.array(avg_score_train_mean))))
        print("Std score test sets mean: {}".format(np.std(np.array(avg_score_test_mean))))

        print("Std precision training sets mean: {}".format(np.std(np.array(avg_precision_train_mean))))
        print("Std precision test sets mean: {}".format(np.std(np.array(avg_precision_test_mean))))

        print("Std recall training sets mean: {}".format(np.std(np.array(avg_recall_train_mean))))
        print("Std recall test sets mean: {}".format(np.std(np.array(avg_recall_test_mean))))

        print("Std F1 training sets mean: {}".format(np.std(np.array(avg_f1_train_mean))))
        print("Std F1 test sets mean: {}".format(np.std(np.array(avg_f1_test_mean))))
        #########################################
        #########################################
        print("")
        print("")
        print("")
        print("")
        ########################################
        ########################################
        print("")
        print("Avg score training sets max: {}".format(np.mean(np.array(avg_score_train_max))))
        print("Avg score test sets max: {}".format(np.mean(np.array(avg_score_test_max))))

        print("Avg precision training sets max: {}".format(np.mean(np.array(avg_precision_train_max))))
        print("Avg precision test sets max: {}".format(np.mean(np.array(avg_precision_test_max))))

        print("Avg recall training sets max: {}".format(np.mean(np.array(avg_recall_train_max))))
        print("Avg recall test sets max: {}".format(np.mean(np.array(avg_recall_test_max))))

        print("Avg F1 training sets max: {}".format(np.mean(np.array(avg_f1_train_max))))
        print("Avg F1 test sets max: {}".format(np.mean(np.array(avg_f1_test_max))))
        print("")
        print("")
        print("Std score training sets max: {}".format(np.std(np.array(avg_score_train_max))))
        print("Std score test sets max: {}".format(np.std(np.array(avg_score_test_max))))

        print("Std precision training sets max: {}".format(np.std(np.array(avg_precision_train_max))))
        print("Std precision test sets max: {}".format(np.std(np.array(avg_precision_test_max))))

        print("Std recall training sets max: {}".format(np.std(np.array(avg_recall_train_max))))
        print("Std recall test sets max: {}".format(np.std(np.array(avg_recall_test_max))))

        print("Std F1 training sets max: {}".format(np.std(np.array(avg_f1_train_max))))
        print("Std F1 test sets max: {}".format(np.std(np.array(avg_f1_test_max))))

        print("####################################################################")
        print("####################################################################")
        print("####################################################################")
        print("####################################################################")

In [ ]:
C_list = [0.0001,0.001,0.01,0.1,1,10,100,1000]

for c in C_list:
        print("c value is: {}".format(c))
        print("gama value is: {}".format(gamma))
        #Training test with support vecto machine.
        fig, [ax_max, ax_mean] = plt.subplots(1, 2, figsize=(18, 11))

        #Initialize lists to manage metrics.
        avg_precision_train_mean = []
        avg_precision_test_mean = []

        avg_recall_train_mean = []
        avg_recall_test_mean = []

        avg_f1_train_mean = []
        avg_f1_test_mean = []

        avg_score_train_mean = []
        avg_score_test_mean = []

        avg_precision_train_max = []
        avg_precision_test_max = []

        avg_recall_train_max = []
        avg_recall_test_max = []

        avg_f1_train_max = []
        avg_f1_test_max = []
        
        avg_score_train_max = []
        avg_score_test_max = []

        
        
        #Lists for ROC curves. 
        tprs_mean = [] 
        aucs_mean = []

        tprs_max = []
        aucs_max = []
        mean_fpr_max = np.linspace(0, 1, 100)
        mean_fpr_mean = np.linspace(0, 1, 100)
        
        
        svm_mean = svm.SVC(C = c, gamma = gamma, max_iter = 500)
        svm_max = svm.SVC(C = c, gamma = gamma, max_iter = 500)


        for i in range(10):
            _seed = i
    
            df_mean['labels'] = label_list
            df_max['labels'] = label_list


            y_mean = df_mean.pop('labels')
            X_mean = df_mean

            y_max = df_max.pop('labels')
            X_max = df_max
    
    
            #Create training and test sets for each fold.
            X_train_mean_index,X_test_mean_index,y_train_mean,y_test_mean = train_test_split(X_mean.index,y_mean,random_state =_seed, test_size=0.4)
            X_train_max_index,X_test_max_index,y_train_max,y_test_max = train_test_split(X_max.index,y_max,random_state =_seed,test_size=0.4)
   
            X_train_mean = X_mean.iloc[X_train_mean_index].to_numpy()
            X_test_mean = X_mean.iloc[X_test_mean_index].to_numpy()
            y_train_mean = y_train_mean.to_numpy()
            y_test_mean = y_test_mean.to_numpy()
    

            X_train_max = X_max.iloc[X_train_max_index].to_numpy()
            X_test_max = X_max.iloc[X_test_max_index].to_numpy()
            y_train_max = y_train_max.to_numpy()
            y_test_max = y_test_max.to_numpy()
        
    
            #MEAN AGGREGATION MODEL: perform svm on mean-aggregated data.
            svm_mean.fit(X_train_mean, y_train_mean)
            y_score = svm_mean.fit(X_train_mean, y_train_mean).decision_function(X_test_mean)
            predictions_train_mean = svm_mean.predict(X_train_mean)
            predictions_test_mean = svm_mean.predict(X_test_mean)
    
            score_training_mean = svm_mean.score(X_train_mean, y_train_mean)
            f1_training_mean = f1_score(y_train_mean, predictions_train_mean, average='binary')
            precision_training_mean = precision_score(y_train_mean, predictions_train_mean, average='binary')
            recall_training_mean = recall_score(y_train_mean, predictions_train_mean, average='binary')
    
            score_test_mean = svm_mean.score(X_test_mean, y_test_mean)
            f1_test_mean = f1_score(y_test_mean, predictions_test_mean, average='binary')
            precision_test_mean =precision_score(y_test_mean, predictions_test_mean, average='binary')
            recall_test_mean = recall_score(y_test_mean, predictions_test_mean, average='binary')
        
            avg_precision_train_mean.append(precision_training_mean)
            avg_precision_test_mean.append(precision_test_mean)

            avg_recall_train_mean.append(recall_training_mean)
            avg_recall_test_mean.append(recall_test_mean)

            avg_f1_train_mean.append(f1_training_mean)
            avg_f1_test_mean.append(f1_test_mean)

            avg_score_train_mean.append(score_training_mean)
            avg_score_test_mean.append(score_test_mean)
    
            #collect data for mean aggregation ROC curve
            viz_mean = RocCurveDisplay.from_estimator(
                svm_mean,
                X_test_mean,
                y_test_mean,
                ax=ax_mean,
                name="ROC fold {}".format(i+1),
                alpha=0.3,
            )
    
            interp_tpr_avg = np.interp(mean_fpr_mean, viz_mean.fpr, viz_mean.tpr)
            interp_tpr_avg[0] = 0.0
            tprs_mean.append(interp_tpr_avg)
            aucs_mean.append(viz_mean.roc_auc)
    
    
            #MAX AGGREGATION MODEL: perform svm on mean-aggregated data.
            svm_max.fit(X_train_max, y_train_max)
            y_score = svm_max.fit(X_train_max, y_train_max).decision_function(X_test_max)
            predictions_train_max = svm_max.predict(X_train_max)
            predictions_test_max = svm_max.predict(X_test_max)

            score_training_max = svm_max.score(X_train_max, y_train_max)
            f1_training_max = f1_score(y_train_max, predictions_train_max, average='binary')
            precision_training_max = precision_score(y_train_max, predictions_train_max, average='binary')
            recall_training_max = recall_score(y_train_max, predictions_train_max, average='binary')

            score_test_max = svm_max.score(X_test_max, y_test_max)
            f1_test_max = f1_score(y_test_max, predictions_test_max, average='binary')
            precision_test_max =precision_score(y_test_max, predictions_test_max, average='binary')
            recall_test_max = recall_score(y_test_max, predictions_test_max, average='binary')

            avg_precision_train_max.append(precision_training_max)
            avg_precision_test_max.append(precision_test_max)

            avg_recall_train_max.append(recall_training_max)
            avg_recall_test_max.append(recall_test_max)

            avg_f1_train_max.append(f1_training_max)
            avg_f1_test_max.append(f1_test_max)

            avg_score_train_max.append(score_training_max)
            avg_score_test_max.append(score_test_max)


            #collect data for max aggregation ROC curve
            viz_max = RocCurveDisplay.from_estimator(
                svm_max,
                X_test_max,
                y_test_max,
                ax=ax_max,
                name="ROC fold {}".format(i+1),
                alpha=0.3,
            )
    

            interp_tpr_max = np.interp(mean_fpr_max, viz_max.fpr, viz_max.tpr)
            interp_tpr_max[0] = 0.0
            tprs_max.append(interp_tpr_max)
            aucs_max.append(viz_max.roc_auc)
    
    
        ############################################    
        ############################################
        ############################################
        #Plot avg ROC curves (max aggregation)
        mean_tpr_max = np.mean(tprs_max, axis=0)
        mean_tpr_max[-1] = 1.0
        mean_auc_max = auc(mean_fpr_max, mean_tpr_max)
        std_auc_max = np.std(aucs_max)

        ax_max.plot(
            mean_fpr_max,
            mean_tpr_max,
            color="b",
            label=r"Max ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_max, std_auc_max),
            lw=2,
            alpha=0.8,
        )    

        ax_max.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_max.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
        )
        ax_max.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)
        ###################
        ##################
        #Plot avg ROC curves (mean aggregation)
        mean_tpr_mean = np.mean(tprs_mean, axis=0)
        mean_tpr_mean[-1] = 1.0
        mean_auc_mean = auc(mean_fpr_mean, mean_tpr_mean)
        std_auc_mean = np.std(aucs_mean)

        ax_mean.plot(
            mean_fpr_mean,
            mean_tpr_mean,
            color="b",
            label=r"Max ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_mean, std_auc_mean),
            lw=2,
            alpha=0.8,
        )    

        ax_mean.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_mean.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
        )
        ax_mean.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)
        #plt.show()


        ax_max.set_title("Receiver Operating Characteristic Curves, Max aggregation.")
        ax_mean.set_title("Receiver Operating Characteristic Curves, Mean aggregation.")

        ax_max.grid(linestyle="--")
        ax_mean.grid(linestyle="--")

        ########################################
        ########################################
        print("Avg score training sets mean: {}".format(np.mean(np.array(avg_score_train_mean))))
        print("Avg score test sets mean: {}".format(np.mean(np.array(avg_score_test_mean))))

        print("Avg precision training sets mean: {}".format(np.mean(np.array(avg_precision_train_mean))))
        print("Avg precision test sets mean: {}".format(np.mean(np.array(avg_precision_test_mean))))

        print("Avg recall training sets mean: {}".format(np.mean(np.array(avg_recall_train_mean))))
        print("Avg recall test sets mean: {}".format(np.mean(np.array(avg_recall_test_mean))))

        print("Avg F1 training sets mean: {}".format(np.mean(np.array(avg_f1_train_mean))))
        print("Avg F1 test sets mean: {}".format(np.mean(np.array(avg_f1_test_mean))))
        print("")
        print("")
        print("Std score training sets mean: {}".format(np.std(np.array(avg_score_train_mean))))
        print("Std score test sets mean: {}".format(np.std(np.array(avg_score_test_mean))))

        print("Std precision training sets mean: {}".format(np.std(np.array(avg_precision_train_mean))))
        print("Std precision test sets mean: {}".format(np.std(np.array(avg_precision_test_mean))))

        print("Std recall training sets mean: {}".format(np.std(np.array(avg_recall_train_mean))))
        print("Std recall test sets mean: {}".format(np.std(np.array(avg_recall_test_mean))))

        print("Std F1 training sets mean: {}".format(np.std(np.array(avg_f1_train_mean))))
        print("Std F1 test sets mean: {}".format(np.std(np.array(avg_f1_test_mean))))
        #########################################
        #########################################
        print("")
        print("")
        print("")
        print("")
        ########################################
        ########################################
        print("")
        print("Avg score training sets max: {}".format(np.mean(np.array(avg_score_train_max))))
        print("Avg score test sets max: {}".format(np.mean(np.array(avg_score_test_max))))

        print("Avg precision training sets max: {}".format(np.mean(np.array(avg_precision_train_max))))
        print("Avg precision test sets max: {}".format(np.mean(np.array(avg_precision_test_max))))

        print("Avg recall training sets max: {}".format(np.mean(np.array(avg_recall_train_max))))
        print("Avg recall test sets max: {}".format(np.mean(np.array(avg_recall_test_max))))

        print("Avg F1 training sets max: {}".format(np.mean(np.array(avg_f1_train_max))))
        print("Avg F1 test sets max: {}".format(np.mean(np.array(avg_f1_test_max))))
        print("")
        print("")
        print("Std score training sets max: {}".format(np.std(np.array(avg_score_train_max))))
        print("Std score test sets max: {}".format(np.std(np.array(avg_score_test_max))))

        print("Std precision training sets max: {}".format(np.std(np.array(avg_precision_train_max))))
        print("Std precision test sets max: {}".format(np.std(np.array(avg_precision_test_max))))

        print("Std recall training sets max: {}".format(np.std(np.array(avg_recall_train_max))))
        print("Std recall test sets max: {}".format(np.std(np.array(avg_recall_test_max))))

        print("Std F1 training sets max: {}".format(np.std(np.array(avg_f1_train_max))))
        print("Std F1 test sets max: {}".format(np.std(np.array(avg_f1_test_max))))

        print("####################################################################")
        print("####################################################################")
        print("####################################################################")
        print("####################################################################")

In [ ]:
#Build a list with the possibile permutations of two layers:
#minimum number of nodes per layer: 32
#maximum number of nodes per layer: 224
#step size is 32.

num_layers = 2
min_nodes_per_layer = 32
max_nodes_per_layer = 224
node_step_size = 32

node_options = list(range(
    min_nodes_per_layer, 
    max_nodes_per_layer + 1, 
    node_step_size
))

two_layer_possibilities = [node_options, node_options]

layer_possibilities = [node_options] * num_layers
layer_node_permutations = list(itertools.product(*layer_possibilities))








models = []

for permutation in layer_node_permutations:
    
        tf.keras.backend.clear_session()
        
        #Build model architecture and print it
        print("permutation is: {}".format(permutation))
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(shape=(7 * 7 * 2048,)))
        
        for nodes_at_layer in permutation:
            model.add(tf.keras.layers.Dense(nodes_at_layer, activation='relu'))
        
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        models.append(model)
        


        
        
        #Training test with MLP.
        fig, [ax_max, ax_mean] = plt.subplots(1, 2, figsize=(18, 11))

        #Initialize lists to manage metrics.
        avg_precision_train_mean = []
        avg_precision_test_mean = []

        avg_recall_train_mean = []
        avg_recall_test_mean = []

        avg_f1_train_mean = []
        avg_f1_test_mean = []

        avg_score_train_mean = []
        avg_score_test_mean = []

        avg_precision_train_max = []
        avg_precision_test_max = []

        avg_recall_train_max = []
        avg_recall_test_max = []

        avg_f1_train_max = []
        avg_f1_test_max = []
        
        avg_score_train_max = []
        avg_score_test_max = []


        #Lists for ROC curves. 
        tprs_mean = [] 
        aucs_mean = []

        tprs_max = []
        aucs_max = []
        mean_fpr_max = np.linspace(0, 1, 100)
        mean_fpr_mean = np.linspace(0, 1, 100)
        
        
        for i in range(10):
            
            print("i value is: {}".format(i))
            _seed = i

    
            df_mean['labels'] = label_list
            df_max['labels'] = label_list


            y_mean = df_mean.pop('labels')
            X_mean = df_mean

            y_max = df_max.pop('labels')
            X_max = df_max
    
    
            #Create training and test sets for each fold.
            X_train_mean_index,X_test_mean_index,y_train_mean,y_test_mean = train_test_split(X_mean.index,y_mean,random_state =_seed, test_size=0.4)
            X_train_max_index,X_test_max_index,y_train_max,y_test_max = train_test_split(X_max.index,y_max,random_state =_seed,test_size=0.4)
    
            X_train_mean = X_mean.iloc[X_train_mean_index].to_numpy()
            X_test_mean = X_mean.iloc[X_test_mean_index].to_numpy()
            y_train_mean = y_train_mean.to_numpy()
            y_test_mean = y_test_mean.to_numpy()
    

            X_train_max = X_max.iloc[X_train_max_index].to_numpy()
            X_test_max = X_max.iloc[X_test_max_index].to_numpy()
            y_train_max = y_train_max.to_numpy()
            y_test_max = y_test_max.to_numpy()
    
            #MEAN AGGREGATION MODEL: perform mlp on mean-aggregated data.
            model.fit(X_train_mean, y_train_mean, epochs=500,verbose = 0)

            predictions_train_mean = model.predict(X_train_mean,batch_size=64, verbose=0)
            predictions_test_mean = model.predict(X_test_mean,batch_size=64, verbose=0)
    
    
            score_training_mean = accuracy_score(y_train_mean, (predictions_train_mean > 0.5).astype("int32"))
            f1_training_mean = f1_score(y_train_mean, (predictions_train_mean > 0.5).astype("int32"), average='binary')
            precision_training_mean = precision_score(y_train_mean, (predictions_train_mean > 0.5).astype("int32"), average='binary')
            recall_training_mean = recall_score(y_train_mean, (predictions_train_mean > 0.5).astype("int32"), average='binary')
    
            score_test_mean = accuracy_score(y_test_mean, (predictions_test_mean > 0.5).astype("int32"))
            f1_test_mean = f1_score(y_test_mean, (predictions_test_mean > 0.5).astype("int32"), average='binary')
            precision_test_mean =precision_score(y_test_mean, (predictions_test_mean > 0.5).astype("int32"), average='binary')
            recall_test_mean = recall_score(y_test_mean, (predictions_test_mean > 0.5).astype("int32"), average='binary')

            avg_f1_values_mean.append([f1_training_mean,f1_test_mean,permutation])

            
            avg_precision_train_mean.append(precision_training_mean)
            avg_precision_test_mean.append(precision_test_mean)

            avg_recall_train_mean.append(recall_training_mean)
            avg_recall_test_mean.append(recall_test_mean)

            avg_f1_train_mean.append(f1_training_mean)
            avg_f1_test_mean.append(f1_test_mean)

            avg_score_train_mean.append(score_training_mean)
            avg_score_test_mean.append(score_test_mean)
    
            #collect data for mean aggregation ROC curve
            viz_mean = RocCurveDisplay.from_predictions(
                y_test_mean,
                (predictions_test_mean > 0.5).astype("int32"),
                ax=ax_mean,
                name="ROC fold {}".format(i+1),
                alpha=0.3
            )
            
            interp_tpr_avg = np.interp(mean_fpr_mean, viz_mean.fpr, viz_mean.tpr)
            interp_tpr_avg[0] = 0.0
            tprs_mean.append(interp_tpr_avg)
            aucs_mean.append(viz_mean.roc_auc)
    
    
            #MAX AGGREGATION MODEL: perform mlp on max-aggregated data.
            model.fit(X_train_max, y_train_max, epochs=500,verbose = 0)
            
            predictions_train_max = model.predict(X_train_max,batch_size=64, verbose=0)
            predictions_test_max = model.predict(X_test_max,batch_size=64, verbose=0)

            score_training_max = accuracy_score(y_train_max, (predictions_train_max > 0.5).astype("int32"))
            f1_training_max = f1_score(y_train_max, (predictions_train_max > 0.5).astype("int32"), average='binary')
            precision_training_max = precision_score(y_train_max, (predictions_train_max > 0.5).astype("int32"), average='binary')
            recall_training_max = recall_score(y_train_max, (predictions_train_max > 0.5).astype("int32"), average='binary')

            score_test_max = accuracy_score(y_test_max, (predictions_test_max > 0.5).astype("int32"))
            f1_test_max = f1_score(y_test_max, (predictions_test_max > 0.5).astype("int32"), average='binary')
            precision_test_max =precision_score(y_test_max, (predictions_test_max > 0.5).astype("int32"), average='binary')
            recall_test_max = recall_score(y_test_max, (predictions_test_max > 0.5).astype("int32"), average='binary')
            

            avg_precision_train_max.append(precision_training_max)
            avg_precision_test_max.append(precision_test_max)

            avg_recall_train_max.append(recall_training_max)
            avg_recall_test_max.append(recall_test_max)

            avg_f1_train_max.append(f1_training_max)
            avg_f1_test_max.append(f1_test_max)

            avg_score_train_max.append(score_training_max)
            avg_score_test_max.append(score_test_max)

            

            #collect data for max aggregation ROC curve
            viz_max = RocCurveDisplay.from_predictions(
                    y_test_max,
                    (predictions_test_max > 0.5).astype("int32"),
                    ax=ax_max,
                    name="ROC fold {}".format(i+1),
                    alpha=0.3,
            )

            interp_tpr_avg = np.interp(mean_fpr_max, viz_max.fpr, viz_max.tpr)
            interp_tpr_avg[0] = 0.0
            tprs_max.append(interp_tpr_avg)
            aucs_max.append(viz_max.roc_auc)


        ############################################    
        ############################################
        ############################################
        #Plot avg ROC curves (max aggregation)
        mean_tpr_max = np.mean(tprs_max, axis=0)
        mean_tpr_max[-1] = 1.0
        mean_auc_max = auc(mean_fpr_max, mean_tpr_max)
        std_auc_max = np.std(aucs_max)

        ax_max.plot(
            mean_fpr_max,
            mean_tpr_max,
            color="b",
            label=r"Avg. ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_max, std_auc_max),
            lw=2,
            alpha=0.8,
        )    

        ax_max.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_max.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
        )
        ax_max.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)
        ##################
        ##################
        #Plot avg ROC curves (mean aggregation)

        mean_tpr_mean = np.mean(tprs_mean, axis=0)
        mean_tpr_mean[-1] = 1.0
        mean_auc_mean = auc(mean_fpr_mean, mean_tpr_mean)
        std_auc_mean = np.std(aucs_mean)

        ax_mean.plot(
            mean_fpr_mean,
            mean_tpr_mean,
            color="b",
            label=r"Max ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc_mean, std_auc_mean),
            lw=2,
            alpha=0.8,
        )    

        ax_mean.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)    

        ax_mean.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic example MAX",
            alpha=0.8,
        )
        ax_mean.legend(loc="lower right")
        plt.rcParams["figure.figsize"] = (20,3)
        #plt.show()


        ax_max.set_title("Receiver Operating Characteristic Curves, Max aggregation, MLP")
        ax_mean.set_title("Receiver Operating Characteristic Curves, Mean aggregation, MLP")

        ax_max.grid(linestyle="--")
        ax_mean.grid(linestyle="--")

        ################
        ################
        ################

        ########################################
        ########################################
        print("Avg score training sets mean: {}".format(np.mean(np.array(avg_score_train_mean))))
        print("Avg score test sets mean: {}".format(np.mean(np.array(avg_score_test_mean))))

        print("Avg precision training sets mean: {}".format(np.mean(np.array(avg_precision_train_mean))))
        print("Avg precision test sets mean: {}".format(np.mean(np.array(avg_precision_test_mean))))

        print("Avg recall training sets mean: {}".format(np.mean(np.array(avg_recall_train_mean))))
        print("Avg recall test sets mean: {}".format(np.mean(np.array(avg_recall_test_mean))))

        print("Avg F1 training sets mean: {}".format(np.mean(np.array(avg_f1_train_mean))))
        print("Avg F1 test sets mean: {}".format(np.mean(np.array(avg_f1_test_mean))))
        print("")
        print("")
        print("Std score training sets mean: {}".format(np.std(np.array(avg_score_train_mean))))
        print("Std score test sets mean: {}".format(np.std(np.array(avg_score_test_mean))))

        print("Std precision training sets mean: {}".format(np.std(np.array(avg_precision_train_mean))))
        print("Std precision test sets mean: {}".format(np.std(np.array(avg_precision_test_mean))))

        print("Std recall training sets mean: {}".format(np.std(np.array(avg_recall_train_mean))))
        print("Std recall test sets mean: {}".format(np.std(np.array(avg_recall_test_mean))))

        print("Std F1 training sets mean: {}".format(np.std(np.array(avg_f1_train_mean))))
        print("Std F1 test sets mean: {}".format(np.std(np.array(avg_f1_test_mean))))
        #########################################
        #########################################
        print("")
        print("")
        print("")
        print("")
        ########################################
        ########################################
        print("")
        print("Avg score training sets max: {}".format(np.mean(np.array(avg_score_train_max))))
        print("Avg score test sets max: {}".format(np.mean(np.array(avg_score_test_max))))

        print("Avg precision training sets max: {}".format(np.mean(np.array(avg_precision_train_max))))
        print("Avg precision test sets max: {}".format(np.mean(np.array(avg_precision_test_max))))

        print("Avg recall training sets max: {}".format(np.mean(np.array(avg_recall_train_max))))
        print("Avg recall test sets max: {}".format(np.mean(np.array(avg_recall_test_max))))

        print("Avg F1 training sets max: {}".format(np.mean(np.array(avg_f1_train_max))))
        print("Avg F1 test sets max: {}".format(np.mean(np.array(avg_f1_test_max))))
        print("")
        print("")
        print("Std score training sets max: {}".format(np.std(np.array(avg_score_train_max))))
        print("Std score test sets max: {}".format(np.std(np.array(avg_score_test_max))))

        print("Std precision training sets max: {}".format(np.std(np.array(avg_precision_train_max))))
        print("Std precision test sets max: {}".format(np.std(np.array(avg_precision_test_max))))

        print("Std recall training sets max: {}".format(np.std(np.array(avg_recall_train_max))))
        print("Std recall test sets max: {}".format(np.std(np.array(avg_recall_test_max))))

        print("Std F1 training sets max: {}".format(np.std(np.array(avg_f1_train_max))))
        print("Std F1 test sets max: {}".format(np.std(np.array(avg_f1_test_max))))

        print("####################################################################")
        print("####################################################################")
        print("####################################################################")
        print("####################################################################")

In [47]:
##############################################################################################
##############################################################################################

In [ ]:
#Patch-level cancer recognition.
#100% and 0% cancer folders are selected.
folder_tumor = ['TCGA-G7-7502-01A-01-BS1','TCGA-DW-7836-01A-01-TS1','TCGA-B8-A54K-01A-01-TSA','TCGA-B9-A8YH-01A-01-TS1'] 
folder_normal = ['TCGA-P4-A5E8-11A-01-TS1','TCGA-P4-A5ED-11A-01-TS1','TCGA-GL-A59R-11A-01-TS1'] 

test_tumor_path = r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\thesis_data_jpg'
test_normal_path = r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\thesis_data_jpg'

#declare lists to store predictions.
prediction_tumor_mlp_list_mean = []
prediction_normal_mlp_list_mean = []

#Build test label list for patch-level classification problem.
test_label_list = []

image_path_test  = r'C:\Users\fabga\OneDrive\Desktop\actual data cancer\thesis_data_jpg'
test_directories = ['TCGA-G7-7502-01A-01-BS1','TCGA-DW-7836-01A-01-TS1','TCGA-B8-A54K-01A-01-TSA','TCGA-B9-A8YH-01A-01-TS1',
                    'TCGA-P4-A5E8-11A-01-TS1','TCGA-P4-A5ED-11A-01-TS1','TCGA-GL-A59R-11A-01-TS1']

j=0
for folder in test_directories:
    #print(folder)
    for element in os.listdir(image_path_test + '\\' + folder):
        #print(len(dirnames))
        j +=1
        if folder in folder_normal:
            label = 0
        elif folder in folder_tumor:
            label = 1
        test_label_list.append(int(label))
        

#Initialize lists containing predictions over folders.         
prediction_normal_mlp_list_mean = []
prediction__mlp_list_mean = []
j = 0

#Initialize lists containing correctly classified tumor and healty tissues. 
correct_tumor  = []
correct_healty = []

#Make predictions on the sleected folders.        
prediction_0_mlp_list_mean = []
prediction_1_mlp_list_mean = []

k = 0

correct_tumor  = []
correct_healty = []


for folder in folder_1:
    print(folder)
    for element in os.listdir(test_1_path + '\\' + folder):
        file_path = test_1_path + '\\' + folder + '\\' + element
        
        image = np.array(preprocess_input(img_to_array(load_img(file_path, target_size=(224,224)))))
        image_batch = np.expand_dims(image, axis = 0) 
        image_features = model.predict(image_batch,batch_size=1)
        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
        predictions_mlp_mean = functional_mlp.predict(image_features)[0][0]
        
        #append results to list
        prediction_1_mlp_list_mean.append((predictions_mlp_mean> 0.5).astype("int32"))
        #print(prediction_1_mlp_list_mean[k])
        #print(test_label_list[k])        
        #print("")
        
        if  prediction_1_mlp_list_mean[k] == test_label_list[k]:
            correct_tumor.append(file_path)
            #print(prediction_1_mlp_list_mean[k])
            #print(test_label_list[k])
            #print("")
        else:
            pass
        
        k+=1
        if k%100 == 0:
            print("k = {}".format(k))
        
print(k)

print("")
print("")
print("###############################################")
print("")
print("")

j = 0
for folder in folder_0:
    for element in os.listdir(test_0_path + '\\' + folder):
        file_path = test_0_path + '\\' + folder + '\\' + element
        image = np.array(preprocess_input(img_to_array(load_img(file_path, target_size=(224,224)))))
        image_batch = np.expand_dims(image, axis = 0) 
        image_features = model.predict(image_batch,batch_size=1)
        image_features = image_features.reshape((image_features.shape[0], 7 * 7 * 2048))
        predictions_mlp_mean = functional_mlp.predict(image_features)[0][0]

        #append results to list
        prediction_0_mlp_list_mean.append((predictions_mlp_mean> 0.5).astype("int32"))
        #print(prediction_0_mlp_list_mean[j])
        #print(test_label_list[j+k])        
        #print("")
        
        if  prediction_0_mlp_list_mean[j] == test_label_list[j+k+1]:
            correct_healty.append(file_path)
            #print(prediction_0_mlp_list_mean[j])
            #print(test_label_list[j+k+1])
            #print("")
            
        else:
            pass
        
        
        j+=1
        if j%100 == 0:
            print("j = {}".format(j))
print(j)

In [ ]:
#Confusion matrix 
titles_options = [
    ("Patch classification, Holdout set Confusion Matrix", None),
]

class_names = ['Normal','Tumor']
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_predictions(
        prediction_mlp_list_mean,
        test_label_list,
        #prediction_mlp_list_mean,
        display_labels=class_names,
        cmap=plt.cm.Reds,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
#Compute metrics for patch level classification.
prediction_mlp_list_mean = prediction_tumor_mlp_list_mean + prediction_normal_mlp_list_mean
accuracy_test_mean = accuracy_score(prediction_mlp_list_mean, test_label_list)
f1_test_mean = f1_score(test_label_list, prediction_mlp_list_mean, average='binary')
precision_test_mean = precision_score(test_label_list, prediction_mlp_list_mean, average='binary')
recall_test_mean = recall_score(test_label_list, prediction_mlp_list_mean, average='binary')
print("Accuracy mean mlp is: {}".format(accuracy_test_mean))
print("F1 mean mlp is: {}".format(f1_test_mean))
print("Precision mean mlp is: {}".format(precision_test_mean))
print("Recall mean mlp is: {}".format(recall_test_mean))

#Confusion matrix
titles_options = [
    ("Patch classification, Holdout set Confusion Matrix", None),
]

class_names = ['Healty','Tumor']
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_predictions(
        prediction_mlp_list_mean,
        test_label_list,
        display_labels=class_names,
        cmap=plt.cm.Reds,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
#Feature activation maps.
#Build a functional model of the MLP

#functional model.
input_ = tf.keras.layers.Input(shape=(7 * 7 * 2048,))
x = tf.keras.layers.Dense(64, activation='relu')(input_)            
x = tf.keras.layers.Dense(224, activation='relu')(x) 
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)  
predictions = x

functional_mlp = Model(inputs=input_, outputs=predictions)
functional_mlp.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#Fit the model on the appropriate data.
functional_mlp.fit(X_mean, y_mean,epochs=500,verbose = 0)

#build Resnet50 model.
model = ResNet50(weights="imagenet",input_shape=[224,224,3], include_top=False)

#Connect resnet to the MLP.
base_model = ResNet50(weights="imagenet",input_shape=[224,224,3], include_top=False)
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = functional_mlp(x)
predictions = x
transfer_model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
import numpy as np

def get_img_array(img, size):
    
    #array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    img=np.resize(img,(size,size,3))#.astype(np.float32)
    array = np.expand_dims(img, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)
    #print(grads)
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
#collect images with and without tumors
correct_tumor_paths = [correct_tumor[100],correct_tumor[20],correct_tumor[101]]
correct_healty_paths = [correct_healty[7],correct_healty[8],correct_healty[9]]


correct_tumor_vectors = []
correct_tumor_labels=[]

for path in correct_tumor_paths:
    print(path)
    img = img_to_array(load_img(path, target_size=(224, 224)))
    image_converted = tf.image.convert_image_dtype(img, tf.float32)
    image_resized = tf.image.resize(image_converted, (IMG_HEIGHT, IMG_WIDTH))
    correct_tumor_vectors.append(image_resized)

    
correct_healty_vectors = []
correct_halty_labels=[]
for path in correct_healty_paths:
    print(path)
    img = img_to_array(load_img(path, target_size=(224, 224)))
    image_converted = tf.image.convert_image_dtype(img, tf.float32)
    image_resized = tf.image.resize(image_converted, (IMG_HEIGHT, IMG_WIDTH))
    correct_healty_vectors.append(image_resized)

In [ ]:
img_size=224
IMG_HEIGHT=224
IMG_WIDTH=224
CHANNELS=3
label = 0


import matplotlib.pyplot as plt
#img_size=128


image_examples=[]
labels_examples=[]
cam_examples=[]


for image in correct_tumor_vectors:
    image_examples.append(image)
    labels_examples.append(label)

    # Prepare image

    #image_converted = tf.image.convert_image_dtype(image, tf.float32)
    # Resize image
    #image_resized = tf.image.resize(image_converted, (IMG_HEIGHT, IMG_WIDTH))
    img_array=tf.expand_dims(image,axis=0)
    print(img_array.shape)
    # Make model
    model = transfer_model

    # Remove last layer's softmax
    model.layers[-1].activation = None

    # Print what the top predicted class is

    preds = (model.predict(img_array)>0.5).astype("int32")[0][0]
    #print("Label: ", label.numpy(), "Predicted:", np.argmax(preds[0]))#decode_predictions(preds, top=1)[0])
    print(preds)
    
    last_conv_layer_name="conv5_block3_3_conv"
    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    cam_examples.append(heatmap)

    # Display last heatmap
    plt.matshow(heatmap)
    plt.show()